In [5]:
import requests
import base64
import time
import pandas as pd
from tqdm import tqdm

client_id = 'da3c40a910c44231aa4c810c1922fdbb'
client_secret = 'c81e0dd2133043e3ad39aea84135d9fe'

def get_spotify_token(client_id, client_secret):
    auth_str = f"{client_id}:{client_secret}"
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()
    token_url = 'https://accounts.spotify.com/api/token'
    response = requests.post(
        token_url,
        data={'grant_type': 'client_credentials'},
        headers={'Authorization': f'Basic {b64_auth_str}'}
    )
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception(f"Failed to get token: {response.status_code}, {response.text}")


In [6]:
def get_track_popularity(track_id, token, retries=3):
    url = f'https://api.spotify.com/v1/tracks/{track_id}'
    headers = {'Authorization': f'Bearer {token}'}

    for attempt in range(retries):
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            return response.json().get('popularity', None)

        elif response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', 5))
            print(f"Rate limited. Sleeping for {retry_after} seconds...")
            time.sleep(retry_after)

        else:
            print(f"Error {response.status_code} on track {track_id}")
            time.sleep(2)
    return None

In [10]:
CHECKPOINT_FILE = 'MusicsUpdated.csv'
data = pd.read_csv(CHECKPOINT_FILE)

if 'popularity' not in data.columns:
    data['popularity'] = pd.NA

token = get_spotify_token(client_id, client_secret)

na_indices = data[data['popularity'].isna()].index[::-1]

for i, idx in tqdm(enumerate(na_indices), total=len(na_indices)):
    track_id = data.at[idx, 'track_id']
    try:
        pop = get_track_popularity(track_id, token)
        data.at[idx, 'popularity'] = pop
    except Exception as e:
        print(f"❌ Stopping due to error: {e}")
        break

    if i % 100 == 0:
        time.sleep(1)

    if i % 800 == 0:
        data.to_csv(CHECKPOINT_FILE, index=False)
        print("💾 Progress saved.")

data.to_csv(CHECKPOINT_FILE, index=False)
print("✅ Done.")


  0%|          | 1/150281 [00:02<116:22:57,  2.79s/it]

💾 Progress saved.


  1%|          | 801/150281 [04:09<43:18:05,  1.04s/it]

💾 Progress saved.


  1%|          | 1601/150281 [08:15<43:12:40,  1.05s/it]

...

💾 Progress saved.


  3%|▎         | 5001/150281 [25:44<24:02:54,  1.68it/s]

Rate limited. Sleeping for 84858 seconds...


  3%|▎         | 5001/150281 [33:15<16:06:16,  2.51it/s]


KeyboardInterrupt: 

In [9]:
data = pd.read_csv(CHECKPOINT_FILE)
data = data[::-1]
data

,track_id,artists,name,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
164682,3LW19oCCXxiqRcnlcQqPr3,9mm Parabellum Bullet,Hide & Seek,131960,0.190,0.9530,8,-4.144,1,0.1310,0.000033,0.051100,0.2030,0.138,136.475,4,16.0
164681,1dxMDGvIYHFYgRvmw1uMHG,アンティック-珈琲店-,Koi no Dependence,243293,0.513,0.9020,4,-3.914,0,0.0530,0.000715,0.001350,0.0571,0.618,109.923,4,0.0
164680,1gXMORZRGA40PE9rDE9cja,coldrain,The Revelation,254826,0.434,0.9750,10,-3.092,0,0.2680,0.000108,0.001410,0.1630,0.282,158.025,4,0.0
164679,3wkdfXGf5JYErW4b35zP2h,ACIDMAN,Colors Of The Wind,275133,0.351,0.6930,0,-6.811,1,0.1200,0.000940,0.000049,0.1920,0.450,200.350,4,12.0
164678,0tt1RdeJX1RyuU4hMEZ19T,アンティック-珈琲店-,Ryusei Rocket,273440,0.438,0.9330,6,-3.062,0,0.1650,0.003120,0.000000,0.1300,0.421,166.956,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,198853,0.618,0.4430,2,-9.681,1,0.0526,0.469000,0.000000,0.0829,0.167,119.949,4,81.0
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,201933,0.266,0.0596,0,-18.515,1,0.0363,0.905000,0.000071,0.1320,0.143,181.740,3,70.0
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,210826,0.438,0.3590,0,-9.734,1,0.0557,0.210000,0.000000,0.1170,0.120,76.332,4,52.0
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,149610,0.420,0.1660,1,-17.235,1,0.0763,0.924000,0.000006,0.1010,0.267,77.489,4,45.0
